In [1]:
import sys
sys.path.append('..')

In [17]:
import torch
import torchvision
import stad

from pathlib import Path
from torch import nn
from torch.utils.data import DataLoader
from stad.datasets.MVTecDataset import MVTecDataset

In [18]:
Y = {}
Y['img_dir'] = Path('/dgx/inoue/data/bottle/train_teacher')
Y['batch_size'] = 1
Y['epochs'] = 1000
Y['device'] = 'cuda:3'

In [19]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        #Encoder Layers
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16,
                               kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 4,
                               kernel_size = 3, padding = 1)
        #Decoder Layers
        self.t_conv1 = nn.ConvTranspose2d(in_channels = 4, out_channels = 16,
                                          kernel_size = 2, stride = 2)
        self.t_conv2 = nn.ConvTranspose2d(in_channels = 16, out_channels = 3,
                                          kernel_size = 2, stride = 2)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        #コメントに28×28のモノクロ画像をi枚を入力した時の次元を示す
        #encode#                          #in  [i, 1, 28, 28] 
        x = self.relu(self.conv1(x))      #out [i, 16, 28, 28]  
        x = self.pool(x)                  #out [i, 16, 14, 14]
        x = self.relu(self.conv2(x))      #out [i, 4, 14, 14]
        x = self.pool(x)                  #out [i ,4, 7, 7]
        #decode#
        x = self.relu(self.t_conv1(x))    #out [i, 16, 14, 14]
        x = self.sigmoid(self.t_conv2(x)) #out [i, 1, 28, 28]
        return x

In [20]:
import albumentations as albu
from albumentations.pytorch import ToTensorV2

train_augs = [albu.HorizontalFlip(p=0.5),
              albu.RandomCrop(height=128, width=128, always_apply=True, p=1)]
train_augs = albu.Compose(train_augs)    

preprocessing = [albu.Normalize(always_apply=True, p=1),
                 ToTensorV2()]
preprocessing = albu.Compose(preprocessing)

In [21]:
mvtec = MVTecDataset(img_dir=Y['img_dir'],
                     augmentations=train_augs,
                     preprocessing=preprocessing)

teacher_train_loader = DataLoader(dataset=mvtec,
                                  batch_size=Y['batch_size'],
                                  shuffle=True)

In [22]:
model = AutoEncoder().to(Y['device'])

In [23]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

In [25]:
for epoch in range(Y['epochs']):
    for img in teacher_train_loader:
        img = img.to(Y['device'])
        output = model(img)
        loss = criterion(output, img)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch%100 == 0:    
        print(f"epoch [{epoch}/{Y['epochs']}]")

epoch [0/1000]


KeyboardInterrupt: 